In [13]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import TimeSeriesSplit


In [14]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
def train_data(name):
    data = all_stock_data[all_stock_data['ticker'] == name]
    ts = data['close']
    train = ts[:int(0.8*(len(ts)))]
    return train

def test_data(name): 
    data = all_stock_data[all_stock_data['ticker'] == name]
    ts = data['close']
    test = ts[int(0.8*(len(ts))):]
    return test

In [15]:
# Lấy data để phục vụ test model 
%run Collect_Data_Function.ipynb
# Lấy danh sách các mã cổ phiếu trên VN30
stock_symbols = ["PHR","MBB"]  # Cột chứa mã cổ phiếu
all_stock_data = data_collect(stock_symbols)
all_stock_data.head()


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


,time,open,high,low,close,volume,ticker
0,2022-01-04,59850,61680,59520,60180,475900,PHR
1,2022-01-05,60360,61360,59190,60930,865700,PHR
2,2022-01-06,61360,65190,61020,65190,1893600,PHR
3,2022-01-07,64950,65450,63440,65110,463800,PHR
4,2022-01-10,65110,67610,64019,65110,694800,PHR


# 1. Linear Regression

Các chỉ báo kỹ thuật là những công cụ được sử dụng trong phân tích thị trường tài chính, giúp nhà đầu tư và các chuyên gia dự đoán hướng đi của giá cổ phiếu hoặc các tài sản khác dựa trên các dữ liệu lịch sử như giá, khối lượng giao dịch, và các thông số kỹ thuật khác. Việc thêm các chỉ báo kỹ thuật giúp mô hình dự báo trở nên mạnh mẽ hơn, nhạy bén hơn trong việc nhận diện các xu hướng và biến động trong thị trường.

1. Đường trung bình động (Moving Average - MA)
+ Ý nghĩa: Đường trung bình động là chỉ báo đơn giản nhất trong phân tích kỹ thuật, giúp làm mượt các biến động giá trong thời gian ngắn hạn để xác định xu hướng chính.
+ Lý do thêm vào: MA giúp xác định xu hướng dài hạn của thị trường. Ví dụ, MA 5 ngày (5-day moving average) giúp theo dõi biến động giá trong 5 ngày gần nhất và dễ dàng phát hiện các xu hướng tăng hoặc giảm.
2. Chỉ số sức mạnh tương đối (Relative Strength Index - RSI)
+ Ý nghĩa: RSI là một chỉ báo dao động (oscillator) dùng để đo lường độ mạnh yếu của một xu hướng giá. RSI có giá trị từ 0 đến 100. Khi RSI vượt qua mức 70, thị trường được coi là quá mua (overbought), và khi RSI dưới mức 30, thị trường được coi là quá bán (oversold).
+ Lý do thêm vào: RSI giúp dự đoán các điểm đảo chiều của thị trường, ví dụ như khi một tài sản có thể đang trong tình trạng quá mua hoặc quá bán và có thể đảo chiều. Điều này giúp nhà đầu tư xác định thời điểm mua hoặc bán hợp lý.
3. Chỉ báo hội tụ phân kỳ trung bình động (MACD)
+ Ý nghĩa: MACD là một chỉ báo dao động dùng để xác định sự thay đổi của xu hướng, đo sự khác biệt giữa đường trung bình động hàm nhanh và chậm. MACD còn bao gồm một đường tín hiệu giúp phát hiện các tín hiệu mua bán.
+ Lý do thêm vào: MACD giúp xác định xu hướng và điểm đảo chiều của thị trường. Khi MACD cắt qua đường tín hiệu từ dưới lên, đó là tín hiệu mua, và khi MACD cắt qua đường tín hiệu từ trên xuống, đó là tín hiệu bán.
4. Dải Bollinger (Bollinger Bands)
+ Ý nghĩa: Dải Bollinger bao gồm một đường trung bình động (thường là MA 20 ngày) và hai dải bao quanh đường trung bình này, được tính bằng độ lệch chuẩn. Dải trên và dưới giúp xác định phạm vi biến động giá của cổ phiếu.
+ Lý do thêm vào: Dải Bollinger giúp xác định mức độ biến động của giá. Nếu giá chạm dải trên, cổ phiếu có thể đang trong tình trạng quá mua (overbought), và nếu giá chạm dải dưới, cổ phiếu có thể đang quá bán (oversold). Điều này có thể giúp nhà đầu tư phát hiện cơ hội đảo chiều.

Tại sao phải thêm các chỉ báo kỹ thuật vào mô hình?

+ Các chỉ báo kỹ thuật không chỉ giúp cung cấp thông tin về tình trạng của thị trường mà còn giúp làm phong phú mô hình dự báo, tạo ra các đặc trưng bổ sung cho mô hình học máy, giúp mô hình hiểu được các xu hướng tiềm ẩn và đưa ra dự báo chính xác hơn.

+ Cải thiện dự báo: Các chỉ báo kỹ thuật cung cấp các thông tin quan trọng về xu hướng giá, mức độ biến động, sức mạnh của xu hướng, v.v. Việc thêm vào các chỉ báo này sẽ giúp mô hình có cái nhìn sâu sắc hơn về dữ liệu và cải thiện khả năng dự báo.

+ Xác định các yếu tố không nhìn thấy trực tiếp từ dữ liệu giá: Dữ liệu giá đơn thuần không thể phản ánh được tất cả các yếu tố ảnh hưởng đến giá cổ phiếu. Chỉ báo kỹ thuật cung cấp thông tin từ các góc độ khác nhau (như xu hướng, mức độ mạnh yếu, biến động) để tăng cường dự báo.

+ Phát hiện tín hiệu mua/bán: Các chỉ báo kỹ thuật giúp phát hiện tín hiệu giao dịch, như mức độ quá mua/quá bán hoặc các điểm đảo chiều, từ đó cải thiện các quyết định giao dịch.


In [17]:
import pandas_ta as ta 
from sklearn.linear_model import LinearRegression

In [18]:
def model_Linear_Regression(name, number_days_predict, n_splits=5):
    # Tải dữ liệu cổ phiếu
    data = all_stock_data[all_stock_data['ticker'] == name]

    # Đường MA 5 ngày
    data['MA_5'] = data['close'].rolling(window=5).mean()  
    
    # Chỉ số RSI
    data['RSI'] = ta.rsi(data['close'], length=14)         

    # MACD
    macd_result = ta.macd(data['close'], fast=12, slow=26, signal=9)
    data['MACD'] = macd_result['MACD_12_26_9']
    data['Signal'] = macd_result['MACDs_12_26_9']

    # Bollinger Bands
    bbands = ta.bbands(data['close'], length=20)
    data['Bollinger_Upper'] = bbands['BBU_20_2.0']
    data['Bollinger_Lower'] = bbands['BBL_20_2.0']

    # Tạo cột Target
    data['Target'] = data['close'].shift(-number_days_predict)

    # Loại bỏ hàng NaN
    data = data.dropna()

    # Xây dựng tập dữ liệu
    X = data[['close', 'MA_5', 'RSI', 'MACD', 'Bollinger_Upper', 'Bollinger_Lower']]
    y = data['Target']

    model = LinearRegression()

    ts_split = TimeSeriesSplit(n_splits=n_splits)

    errors = []

    for train_index, test_index in ts_split.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        model.fit(X_train, y_train)
        
        y_pred = model.predict(X_test)
        
        mape = mean_absolute_percentage_error(y_test, y_pred)
        errors.append(mape)
    
    avg_mape = sum(errors) / len(errors)

    # Dự báo cho n ngày tới
    Future_Price = model.predict(X.tail(number_days_predict))

    return Future_Price[-1], avg_mape, y_pred

In [19]:
# Future_Price_LR,mape_LR,y_pred_LR= model_Linear_Regression("PHR",2)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

# 2. Ridge, Lasso, ElasticNet Regression

In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge, Lasso, ElasticNet

In [21]:
def model_RLE(name, number_days_predict, n_splits=5, model_type='ridge'):
    
    # Kiểm tra đầu vào
    if model_type not in ['ridge', 'lasso', 'elasticnet']:
        raise ValueError("model_type phải là 'ridge', 'lasso', hoặc 'elasticnet'.")

    # Lấy dữ liệu
    data = all_stock_data[all_stock_data['ticker'] == name]

    # Thêm các chỉ báo kỹ thuật
    data['MA_5'] = data['close'].rolling(window=5).mean()
    data['RSI'] = ta.rsi(data['close'], length=14)

    macd_result = ta.macd(data['close'], fast=12, slow=26, signal=9)
    data['MACD'] = macd_result['MACD_12_26_9']
    data['Signal'] = macd_result['MACDs_12_26_9']

    bbands = ta.bbands(data['close'], length=20)
    data['Bollinger_Upper'] = bbands['BBU_20_2.0']
    data['Bollinger_Lower'] = bbands['BBL_20_2.0']

    # Tạo cột Target
    data['Target'] = data['close'].shift(-number_days_predict)
    data = data.dropna()

    # Dữ liệu đặc trưng và mục tiêu
    X = data[['close', 'MA_5', 'RSI', 'MACD', 'Bollinger_Upper', 'Bollinger_Lower']]
    y = data['Target']

    # Thay thế NaN trong X
    X = X.fillna(method='ffill').fillna(method='bfill')

    # Khởi tạo TimeSeriesSplit
    tscv = TimeSeriesSplit(n_splits=n_splits)

    # Chọn mô hình và tham số
    if model_type == 'ridge':
        model = Ridge()
        param_grid = {'alpha': [0.01, 0.1, 1, 10, 100]}
    elif model_type == 'lasso':
        model = Lasso(max_iter=10000)
        param_grid = {'alpha': [0.01, 0.1, 1, 10, 100]}
    elif model_type == 'elasticnet':
        model = ElasticNet(max_iter=10000)
        param_grid = {
            'alpha': [0.01, 0.1, 1, 10, 100],
            'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]
        }

    # GridSearchCV với TimeSeriesSplit
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=tscv)
    grid_search.fit(X, y)

    # Lấy mô hình tốt nhất
    best_model = grid_search.best_estimator_

    # Đánh giá mô hình trên tập kiểm tra
    train_idx, test_idx = list(tscv.split(X))[-1]
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    best_model.fit(X_train, y_train)
    y_pred = best_model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)

    # Dự báo n ngày tới sau khi đánh giá
    last_data = X.iloc[-1:].copy()
    Future_Price = []

    for _ in range(number_days_predict):
        future_pred = best_model.predict(last_data)[0]
        Future_Price.append(future_pred)

        # Cập nhật dữ liệu đặc trưng cho ngày tiếp theo
        new_row = last_data.copy()
        new_row['close'] = future_pred
        new_row['MA_5'] = np.mean([future_pred] + last_data['close'].tail(4).tolist())

        # Cập nhật RSI
        temp_close = pd.Series(last_data['close'].tolist() + [future_pred])
        new_row['RSI'] = ta.rsi(temp_close, length=14).iloc[-1] if len(temp_close) >= 14 else last_data['RSI'].iloc[-1]

        # Cập nhật MACD và Bollinger Bands
        temp_close = pd.Series(last_data['close'].tolist() + [future_pred])
        macd_result = ta.macd(temp_close, fast=12, slow=26, signal=9)
        new_row['MACD'] = macd_result['MACD_12_26_9'].iloc[-1] if len(temp_close) >= 26 else last_data['MACD'].iloc[-1]
        bbands = ta.bbands(temp_close, length=20)
        new_row['Bollinger_Upper'] = bbands['BBU_20_2.0'].iloc[-1] if len(temp_close) >= 20 else last_data['Bollinger_Upper'].iloc[-1]
        new_row['Bollinger_Lower'] = bbands['BBL_20_2.0'].iloc[-1] if len(temp_close) >= 20 else last_data['Bollinger_Lower'].iloc[-1]

        last_data = new_row  # Cập nhật hàng mới

    return Future_Price[-1], mape, y_pred


In [22]:
# Future_Price_Ridge, mape_Ridge, y_pred_Ridge = model_RLE('PHR',7)
# print(Future_Price_Ridge)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

52995.30096051281


In [23]:
# Future_Price_Lesso, mape_Lesso, y_pred_Lesso = model_RLE('PHR',7,model_type='lasso')
# print(Future_Price_Lesso)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

53076.15123841593


In [24]:
# Future_Price_ElasticNet, mape_ElasticNet, y_pred_ElasticNet = model_RLE('PHR',7,model_type='elasticnet')
# print(Future_Price_ElasticNet)


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

54991.84597649772


# 3. Support Vector Machine (SVM)

In [25]:
from sklearn.svm import SVR

In [26]:
def model_SVR(name, number_days_predict, n_splits=5):
    # Lấy dữ liệu cổ phiếu
    data = all_stock_data[all_stock_data['ticker'] == name]

    # Thêm các chỉ báo kỹ thuật
    data['MA_5'] = data['close'].rolling(window=5).mean()
    data['RSI'] = ta.rsi(data['close'], length=14)

    macd_result = ta.macd(data['close'], fast=12, slow=26, signal=9)
    data['MACD'] = macd_result['MACD_12_26_9']
    data['Signal'] = macd_result['MACDs_12_26_9']

    bbands = ta.bbands(data['close'], length=20)
    data['Bollinger_Upper'] = bbands['BBU_20_2.0']
    data['Bollinger_Lower'] = bbands['BBL_20_2.0']

    # Tạo cột Target
    data['Target'] = data['close'].shift(-number_days_predict)
    data = data.dropna()

    # Xây dựng tập dữ liệu
    X = data[['close', 'MA_5', 'RSI', 'MACD', 'Bollinger_Upper', 'Bollinger_Lower']]
    y = data['Target']

    # Thay thế NaN trong X
    X = X.fillna(method='ffill').fillna(method='bfill')

    # Time Series Split
    tscv = TimeSeriesSplit(n_splits=n_splits)

    # Khởi tạo SVR và GridSearchCV
    svr = SVR()
    param_grid = {
        'kernel': ['linear', 'rbf', 'poly'],
        'C': [0.1, 1, 10, 100],
        'gamma': ['scale', 'auto'],
        'epsilon': [0.1, 0.2, 0.5]
    }

    grid_search = GridSearchCV(estimator=svr, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=tscv)
    grid_search.fit(X, y)

    # Lấy mô hình tốt nhất
    best_model = grid_search.best_estimator_

    # Đánh giá trên tập kiểm tra
    train_idx, test_idx = list(tscv.split(X))[-1]
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    best_model.fit(X_train, y_train)
    y_pred = best_model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)

    # Dự báo n ngày tới
    last_data = X.iloc[-1:].copy()
    future_predictions = []

    for _ in range(number_days_predict):
        future_pred = best_model.predict(last_data)[0]
        future_predictions.append(future_pred)

        # Cập nhật dữ liệu đặc trưng cho ngày tiếp theo
        new_row = last_data.copy()
        new_row['close'] = future_pred
        new_row['MA_5'] = np.mean([future_pred] + last_data['close'].tail(4).tolist())

        temp_close = pd.Series(last_data['close'].tolist() + [future_pred])
        new_row['RSI'] = ta.rsi(temp_close, length=14).iloc[-1] if len(temp_close) >= 14 else last_data['RSI'].iloc[-1]
        macd_result = ta.macd(temp_close, fast=12, slow=26, signal=9)
        new_row['MACD'] = macd_result['MACD_12_26_9'].iloc[-1] if len(temp_close) >= 26 else last_data['MACD'].iloc[-1]
        bbands = ta.bbands(temp_close, length=20)
        new_row['Bollinger_Upper'] = bbands['BBU_20_2.0'].iloc[-1] if len(temp_close) >= 20 else last_data['Bollinger_Upper'].iloc[-1]
        new_row['Bollinger_Lower'] = bbands['BBL_20_2.0'].iloc[-1] if len(temp_close) >= 20 else last_data['Bollinger_Lower'].iloc[-1]

        last_data = new_row

    return future_predictions, mape, y_pred

In [ ]:
future_prices, mape_svr, y_pred_svr = model_SVR('PHR', 7)
print("Dự báo 7 ngày tới:", future_prices)
print("MAPE:", mape_svr)



A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/

# 4. Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
def model_RandomForest(name, number_days_predict, n_splits=5):
    # Lấy dữ liệu cổ phiếu
    data = all_stock_data[all_stock_data['ticker'] == name]

    # Thêm các chỉ báo kỹ thuật
    data['MA_5'] = data['close'].rolling(window=5).mean()
    data['RSI'] = ta.rsi(data['close'], length=14)

    macd_result = ta.macd(data['close'], fast=12, slow=26, signal=9)
    data['MACD'] = macd_result['MACD_12_26_9']
    data['Signal'] = macd_result['MACDs_12_26_9']

    bbands = ta.bbands(data['close'], length=20)
    data['Bollinger_Upper'] = bbands['BBU_20_2.0']
    data['Bollinger_Lower'] = bbands['BBL_20_2.0']

    # Tạo cột Target
    data['Target'] = data['close'].shift(-number_days_predict)
    data = data.dropna()

    # Xây dựng tập dữ liệu
    X = data[['close', 'MA_5', 'RSI', 'MACD', 'Bollinger_Upper', 'Bollinger_Lower']]
    y = data['Target']

    # Thay thế NaN trong X
    X = X.fillna(method='ffill').fillna(method='bfill')

    # Time Series Split
    tscv = TimeSeriesSplit(n_splits=n_splits)

    # Khởi tạo Random Forest và GridSearchCV
    rf = RandomForestRegressor(random_state=42)
    param_grid = {
        'n_estimators': [50, 100, 200],
        'max_depth': [5, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=tscv)
    grid_search.fit(X, y)

    # Lấy mô hình tốt nhất
    best_model = grid_search.best_estimator_

    # Đánh giá trên tập kiểm tra
    train_idx, test_idx = list(tscv.split(X))[-1]
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    best_model.fit(X_train, y_train)
    y_pred = best_model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)

    # Dự báo n ngày tới
    last_data = X.iloc[-1:].copy()
    future_predictions = []

    for _ in range(number_days_predict):
        future_pred = best_model.predict(last_data)[0]
        future_predictions.append(future_pred)

        # Cập nhật dữ liệu đặc trưng cho ngày tiếp theo
        new_row = last_data.copy()
        new_row['close'] = future_pred
        new_row['MA_5'] = np.mean([future_pred] + last_data['close'].tail(4).tolist())

        temp_close = pd.Series(last_data['close'].tolist() + [future_pred])
        new_row['RSI'] = ta.rsi(temp_close, length=14).iloc[-1] if len(temp_close) >= 14 else last_data['RSI'].iloc[-1]
        macd_result = ta.macd(temp_close, fast=12, slow=26, signal=9)
        new_row['MACD'] = macd_result['MACD_12_26_9'].iloc[-1] if len(temp_close) >= 26 else last_data['MACD'].iloc[-1]
        bbands = ta.bbands(temp_close, length=20)
        new_row['Bollinger_Upper'] = bbands['BBU_20_2.0'].iloc[-1] if len(temp_close) >= 20 else last_data['Bollinger_Upper'].iloc[-1]
        new_row['Bollinger_Lower'] = bbands['BBL_20_2.0'].iloc[-1] if len(temp_close) >= 20 else last_data['Bollinger_Lower'].iloc[-1]

        last_data = new_row

    return future_predictions, mape, y_pred

In [ ]:
future_prices, mape_rf, y_pred_rf = model_RandomForest('PHR', 7)
print("Dự báo 7 ngày tới:", future_prices)
print("MAPE:", mape_rf)

# 5. Gradient Boosting Machines (GBM) như XGBoost, LightGBM, CatBoost

In [ ]:
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor

In [ ]:
def model_GBM(name, number_days_predict, model_type='xgboost', n_splits=5):
    # Lấy dữ liệu cổ phiếu
    data = all_stock_data[all_stock_data['ticker'] == name]

    # Thêm các chỉ báo kỹ thuật
    data['MA_5'] = data['close'].rolling(window=5).mean()
    data['RSI'] = ta.rsi(data['close'], length=14)

    macd_result = ta.macd(data['close'], fast=12, slow=26, signal=9)
    data['MACD'] = macd_result['MACD_12_26_9']
    data['Signal'] = macd_result['MACDs_12_26_9']

    bbands = ta.bbands(data['close'], length=20)
    data['Bollinger_Upper'] = bbands['BBU_20_2.0']
    data['Bollinger_Lower'] = bbands['BBL_20_2.0']

    # Tạo cột Target
    data['Target'] = data['close'].shift(-number_days_predict)
    data = data.dropna()

    # Xây dựng tập dữ liệu
    X = data[['close', 'MA_5', 'RSI', 'MACD', 'Bollinger_Upper', 'Bollinger_Lower']]
    y = data['Target']

    # Thay thế NaN trong X
    X = X.fillna(method='ffill').fillna(method='bfill')

    # Time Series Split
    tscv = TimeSeriesSplit(n_splits=n_splits)

    # Chọn mô hình GBM
    if model_type == 'xgboost':
        model = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)
        param_grid = {
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 5, 7],
            'learning_rate': [0.01, 0.1, 0.2],
            'subsample': [0.8, 1.0]
        }
    elif model_type == 'lightgbm':
        model = lgb.LGBMRegressor(random_state=42)
        param_grid = {
            'n_estimators': [50, 100, 200],
            'max_depth': [-1, 5, 10],
            'learning_rate': [0.01, 0.1, 0.2],
            'num_leaves': [31, 50, 100]
        }
    elif model_type == 'catboost':
        model = CatBoostRegressor(verbose=0, random_state=42)
        param_grid = {
            'iterations': [100, 200, 300],
            'depth': [3, 6, 10],
            'learning_rate': [0.01, 0.1, 0.2]
        }
    else:
        raise ValueError("model_type phải là 'xgboost', 'lightgbm', hoặc 'catboost'.")

    # GridSearchCV với Time Series Split
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=tscv)
    grid_search.fit(X, y)

    # Lấy mô hình tốt nhất
    best_model = grid_search.best_estimator_

    # Đánh giá trên tập kiểm tra
    train_idx, test_idx = list(tscv.split(X))[-1]
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    best_model.fit(X_train, y_train)
    y_pred = best_model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)

    # Dự báo n ngày tới
    last_data = X.iloc[-1:].copy()
    future_predictions = []

    for _ in range(number_days_predict):
        future_pred = best_model.predict(last_data)[0]
        future_predictions.append(future_pred)

        # Cập nhật dữ liệu đặc trưng cho ngày tiếp theo
        new_row = last_data.copy()
        new_row['close'] = future_pred
        new_row['MA_5'] = np.mean([future_pred] + last_data['close'].tail(4).tolist())

        temp_close = pd.Series(last_data['close'].tolist() + [future_pred])
        new_row['RSI'] = ta.rsi(temp_close, length=14).iloc[-1] if len(temp_close) >= 14 else last_data['RSI'].iloc[-1]
        macd_result = ta.macd(temp_close, fast=12, slow=26, signal=9)
        new_row['MACD'] = macd_result['MACD_12_26_9'].iloc[-1] if len(temp_close) >= 26 else last_data['MACD'].iloc[-1]
        bbands = ta.bbands(temp_close, length=20)
        new_row['Bollinger_Upper'] = bbands['BBU_20_2.0'].iloc[-1] if len(temp_close) >= 20 else last_data['Bollinger_Upper'].iloc[-1]
        new_row['Bollinger_Lower'] = bbands['BBL_20_2.0'].iloc[-1] if len(temp_close) >= 20 else last_data['Bollinger_Lower'].iloc[-1]

        last_data = new_row

    return future_predictions, mape, y_pred

In [ ]:
future_prices_xgb, mape_xgb, y_pred_xgb = model_GBM('PHR', 7, model_type='xgboost')
future_prices_lgb, mape_lgb, y_pred_lgb = model_GBM('PHR', 7, model_type='lightgbm')
future_prices_cb, mape_cb, y_pred_cb = model_GBM('PHR', 7, model_type='catboost')

print("Dự báo 7 ngày tới (XGBoost):", future_prices_xgb)
print("MAPE (XGBoost):", mape_xgb)
print("Dự báo 7 ngày tới (LightGBM):", future_prices_lgb)
print("MAPE (LightGBM):", mape_lgb)
print("Dự báo 7 ngày tới (CatBoost):", future_prices_cb)
print("MAPE (CatBoost):", mape_cb)


# 6. k-Nearest Neighbors (k-NN)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
def model_kNN(name, number_days_predict, n_splits=5):
    # Lấy dữ liệu cổ phiếu
    data = all_stock_data[all_stock_data['ticker'] == name]

    # Thêm các chỉ báo kỹ thuật
    data['MA_5'] = data['close'].rolling(window=5).mean()
    data['RSI'] = ta.rsi(data['close'], length=14)

    macd_result = ta.macd(data['close'], fast=12, slow=26, signal=9)
    data['MACD'] = macd_result['MACD_12_26_9']
    data['Signal'] = macd_result['MACDs_12_26_9']

    bbands = ta.bbands(data['close'], length=20)
    data['Bollinger_Upper'] = bbands['BBU_20_2.0']
    data['Bollinger_Lower'] = bbands['BBL_20_2.0']

    # Tạo cột Target
    data['Target'] = data['close'].shift(-number_days_predict)
    data = data.dropna()

    # Xây dựng tập dữ liệu
    X = data[['close', 'MA_5', 'RSI', 'MACD', 'Bollinger_Upper', 'Bollinger_Lower']]
    y = data['Target']

    # Thay thế NaN trong X
    X = X.fillna(method='ffill').fillna(method='bfill')

    # Time Series Split
    tscv = TimeSeriesSplit(n_splits=n_splits)

    # Mô hình k-NN
    model = KNeighborsRegressor()
    param_grid = {
        'n_neighbors': [3, 5, 10, 20],
        'weights': ['uniform', 'distance'],
        'p': [1, 2]  # 1: Manhattan, 2: Euclidean
    }

    # GridSearchCV với Time Series Split
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='neg_mean_absolute_error', cv=tscv)
    grid_search.fit(X, y)

    # Lấy mô hình tốt nhất
    best_model = grid_search.best_estimator_

    # Đánh giá trên tập kiểm tra
    train_idx, test_idx = list(tscv.split(X))[-1]
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    best_model.fit(X_train, y_train)
    y_pred = best_model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred)

    # Dự báo n ngày tới
    last_data = X.iloc[-1:].copy()
    future_predictions = []

    for _ in range(number_days_predict):
        future_pred = best_model.predict(last_data)[0]
        future_predictions.append(future_pred)

        # Cập nhật dữ liệu đặc trưng cho ngày tiếp theo
        new_row = last_data.copy()
        new_row['close'] = future_pred
        new_row['MA_5'] = np.mean([future_pred] + last_data['close'].tail(4).tolist())

        temp_close = pd.Series(last_data['close'].tolist() + [future_pred])
        new_row['RSI'] = ta.rsi(temp_close, length=14).iloc[-1] if len(temp_close) >= 14 else last_data['RSI'].iloc[-1]
        macd_result = ta.macd(temp_close, fast=12, slow=26, signal=9)
        new_row['MACD'] = macd_result['MACD_12_26_9'].iloc[-1] if len(temp_close) >= 26 else last_data['MACD'].iloc[-1]
        bbands = ta.bbands(temp_close, length=20)
        new_row['Bollinger_Upper'] = bbands['BBU_20_2.0'].iloc[-1] if len(temp_close) >= 20 else last_data['Bollinger_Upper'].iloc[-1]
        new_row['Bollinger_Lower'] = bbands['BBL_20_2.0'].iloc[-1] if len(temp_close) >= 20 else last_data['Bollinger_Lower'].iloc[-1]

        last_data = new_row

    return future_predictions, mape, y_pred

In [ ]:
future_prices_knn, mape_knn, y_pred_knn = model_kNN('PHR', 7)

print("Dự báo 7 ngày tới (k-NN):", future_prices_knn)
print("MAPE (k-NN):", mape_knn)